In [1]:
import pyspark
import findspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import numpy as np

In [5]:
findspark.init()
findspark.find()

'E:\\softwares\\Spark\\spark-3.0.0-bin-hadoop2.7'

In [9]:
if __name__=='__main__':
    spark=SparkSession\
    .builder\
    .appName('LinearRegWithSpark')\
    .getOrCreate()

In [11]:
dataset=spark.read.csv(r"C:\Users\Mohit\Desktop\Sample Files\titanic\train.csv",header=True)

In [12]:
dataset.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [13]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [24]:
from pyspark.sql.functions import col,count,isnan,when
new_data=dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [16]:
new_data.printSchema()

root
 |-- PassengerId: float (nullable = true)
 |-- Survived: float (nullable = true)
 |-- Pclass: float (nullable = true)
 |-- Name: float (nullable = true)
 |-- Sex: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: float (nullable = true)
 |-- Parch: float (nullable = true)
 |-- Ticket: float (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: float (nullable = true)
 |-- Embarked: float (nullable = true)



In [17]:
new_data.show()

+-----------+--------+------+----+----+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name| Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+----+----+----+-----+-----+--------+-------+-----+--------+
|        1.0|     0.0|   3.0|null|null|22.0|  1.0|  0.0|    null|   7.25| null|    null|
|        2.0|     1.0|   1.0|null|null|38.0|  1.0|  0.0|    null|71.2833| null|    null|
|        3.0|     1.0|   3.0|null|null|26.0|  0.0|  0.0|    null|  7.925| null|    null|
|        4.0|     1.0|   1.0|null|null|35.0|  1.0|  0.0|113803.0|   53.1| null|    null|
|        5.0|     0.0|   3.0|null|null|35.0|  0.0|  0.0|373450.0|   8.05| null|    null|
|        6.0|     0.0|   3.0|null|null|null|  0.0|  0.0|330877.0| 8.4583| null|    null|
|        7.0|     0.0|   1.0|null|null|54.0|  0.0|  0.0| 17463.0|51.8625| null|    null|
|        8.0|     0.0|   3.0|null|null| 2.0|  3.0|  1.0|349909.0| 21.075| null|    null|
|        9.0|     1.0

In [19]:
list((col(c).cast("float").alias(c) for c in dataset.columns))

[Column<b'CAST(PassengerId AS FLOAT) AS `PassengerId`'>,
 Column<b'CAST(Survived AS FLOAT) AS `Survived`'>,
 Column<b'CAST(Pclass AS FLOAT) AS `Pclass`'>,
 Column<b'CAST(Name AS FLOAT) AS `Name`'>,
 Column<b'CAST(Sex AS FLOAT) AS `Sex`'>,
 Column<b'CAST(Age AS FLOAT) AS `Age`'>,
 Column<b'CAST(SibSp AS FLOAT) AS `SibSp`'>,
 Column<b'CAST(Parch AS FLOAT) AS `Parch`'>,
 Column<b'CAST(Ticket AS FLOAT) AS `Ticket`'>,
 Column<b'CAST(Fare AS FLOAT) AS `Fare`'>,
 Column<b'CAST(Cabin AS FLOAT) AS `Cabin`'>,
 Column<b'CAST(Embarked AS FLOAT) AS `Embarked`'>]

In [37]:
from pyspark.ml.feature import Imputer

In [25]:
new_data.select([count(when(col(c).isNull(),c)).alias(c) for c in new_data.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0| 891|891|177|    0|    0|   230|   0|  891|     891|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [44]:
imputer=Imputer(inputCols=['PassengerId','Survived','Age','Ticket','Fare'],
               outputCols=['PassengerId','Survived','Age','Ticket','Fare'])

In [45]:
model=imputer.fit(new_data)

In [46]:
imputed_data=model.transform(new_data)

In [47]:
imputed_data.select([count(when(col(c).isNull(),c)).alias(c) for c in new_data.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0| 891|891|  0|    0|    0|     0|   0|  891|     891|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [101]:
features=imputed_data.select('PassengerId','Survived','Age','SibSp','Parch','Fare')

In [99]:
features.show()

+-----------+--------+---------+-----+-----+-------+
|PassengerId|Survived|      Age|SibSp|Parch|   Fare|
+-----------+--------+---------+-----+-----+-------+
|        1.0|     0.0|     22.0|  1.0|  0.0|   7.25|
|        2.0|     1.0|     38.0|  1.0|  0.0|71.2833|
|        3.0|     1.0|     26.0|  0.0|  0.0|  7.925|
|        4.0|     1.0|     35.0|  1.0|  0.0|   53.1|
|        5.0|     0.0|     35.0|  0.0|  0.0|   8.05|
|        6.0|     0.0|29.699118|  0.0|  0.0| 8.4583|
|        7.0|     0.0|     54.0|  0.0|  0.0|51.8625|
|        8.0|     0.0|      2.0|  3.0|  1.0| 21.075|
|        9.0|     1.0|     27.0|  0.0|  2.0|11.1333|
|       10.0|     1.0|     14.0|  1.0|  0.0|30.0708|
|       11.0|     1.0|      4.0|  1.0|  1.0|   16.7|
|       12.0|     1.0|     58.0|  0.0|  0.0|  26.55|
|       13.0|     0.0|     20.0|  0.0|  0.0|   8.05|
|       14.0|     0.0|     39.0|  1.0|  5.0| 31.275|
|       15.0|     0.0|     14.0|  0.0|  0.0| 7.8542|
|       16.0|     1.0|     55.0|  0.0|  0.0|  

In [102]:
assembler=VectorAssembler(
            inputCols=features.columns,
            outputCol='features')

In [103]:
output=assembler.transform(imputed_data)

In [85]:
output.show()
# imputed_data.show()

+-----------+--------+------+----+----+---------+-----+-----+---------+-------+-----+--------+--------------------+
|PassengerId|Survived|Pclass|Name| Sex|      Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|            features|
+-----------+--------+------+----+----+---------+-----+-----+---------+-------+-----+--------+--------------------+
|        1.0|     0.0|   3.0|null|null|     22.0|  1.0|  0.0|260318.55|   7.25| null|    null|[1.0,3.0,22.0,1.0...|
|        2.0|     1.0|   1.0|null|null|     38.0|  1.0|  0.0|260318.55|71.2833| null|    null|[2.0,1.0,38.0,1.0...|
|        3.0|     1.0|   3.0|null|null|     26.0|  0.0|  0.0|260318.55|  7.925| null|    null|[3.0,3.0,26.0,0.0...|
|        4.0|     1.0|   1.0|null|null|     35.0|  1.0|  0.0| 113803.0|   53.1| null|    null|[4.0,1.0,35.0,1.0...|
|        5.0|     0.0|   3.0|null|null|     35.0|  0.0|  0.0| 373450.0|   8.05| null|    null|[5.0,3.0,35.0,0.0...|
|        6.0|     0.0|   3.0|null|null|29.699118|  0.0|  0.0| 330877.0| 

In [104]:
target_df=output.select('features','Survived')

In [93]:
target_df.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[1.0,3.0,22.0,1.0...|     0.0|
|[2.0,1.0,38.0,1.0...|     1.0|
|[3.0,3.0,26.0,0.0...|     1.0|
|[4.0,1.0,35.0,1.0...|     1.0|
|[5.0,3.0,35.0,0.0...|     0.0|
|[6.0,3.0,29.69911...|     0.0|
|[7.0,1.0,54.0,0.0...|     0.0|
|[8.0,3.0,2.0,3.0,...|     0.0|
|[9.0,3.0,27.0,0.0...|     1.0|
|[10.0,2.0,14.0,1....|     1.0|
|[11.0,3.0,4.0,1.0...|     1.0|
|[12.0,1.0,58.0,0....|     1.0|
|[13.0,3.0,20.0,0....|     0.0|
|[14.0,3.0,39.0,1....|     0.0|
|[15.0,3.0,14.0,0....|     0.0|
|[16.0,2.0,55.0,0....|     1.0|
|[17.0,3.0,2.0,4.0...|     0.0|
|[18.0,2.0,29.6991...|     1.0|
|[19.0,3.0,31.0,1....|     0.0|
|[20.0,3.0,29.6991...|     1.0|
+--------------------+--------+
only showing top 20 rows



In [105]:
target_df=target_df.select("features","Survived")

In [76]:
target_df.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[1.0,0.0,3.0,22.0...|     0.0|
|[2.0,1.0,1.0,38.0...|     1.0|
|[3.0,1.0,3.0,26.0...|     1.0|
|[4.0,1.0,1.0,35.0...|     1.0|
|[5.0,0.0,3.0,35.0...|     0.0|
|[6.0,0.0,3.0,29.6...|     0.0|
|[7.0,0.0,1.0,54.0...|     0.0|
|[8.0,0.0,3.0,2.0,...|     0.0|
|[9.0,1.0,3.0,27.0...|     1.0|
|[10.0,1.0,2.0,14....|     1.0|
|[11.0,1.0,3.0,4.0...|     1.0|
|[12.0,1.0,1.0,58....|     1.0|
|[13.0,0.0,3.0,20....|     0.0|
|[14.0,0.0,3.0,39....|     0.0|
|[15.0,0.0,3.0,14....|     0.0|
|[16.0,1.0,2.0,55....|     1.0|
|[17.0,0.0,3.0,2.0...|     0.0|
|[18.0,1.0,2.0,29....|     1.0|
|[19.0,0.0,3.0,31....|     0.0|
|[20.0,1.0,3.0,29....|     1.0|
+--------------------+--------+
only showing top 20 rows



In [106]:
train_df,test_df=target_df.randomSplit([0.7,0.3])

In [95]:
train_df.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[1.0,3.0,22.0,1.0...|     0.0|
|[2.0,1.0,38.0,1.0...|     1.0|
|[3.0,3.0,26.0,0.0...|     1.0|
|[6.0,3.0,29.69911...|     0.0|
|[9.0,3.0,27.0,0.0...|     1.0|
|[11.0,3.0,4.0,1.0...|     1.0|
|[13.0,3.0,20.0,0....|     0.0|
|[16.0,2.0,55.0,0....|     1.0|
|[18.0,2.0,29.6991...|     1.0|
|[19.0,3.0,31.0,1....|     0.0|
|[22.0,2.0,34.0,0....|     1.0|
|[23.0,3.0,15.0,0....|     1.0|
|[25.0,3.0,8.0,3.0...|     0.0|
|[26.0,3.0,38.0,1....|     1.0|
|[27.0,3.0,29.6991...|     0.0|
|[28.0,1.0,19.0,3....|     0.0|
|[29.0,3.0,29.6991...|     1.0|
|[30.0,3.0,29.6991...|     0.0|
|[32.0,1.0,29.6991...|     1.0|
|[33.0,3.0,29.6991...|     1.0|
+--------------------+--------+
only showing top 20 rows



In [124]:
lin_reg=LinearRegression(featuresCol='features',labelCol='Survived',predictionCol='prediction',maxIter=1000)
linear_model=lin_reg.fit(train_df)

In [125]:
print("Coefficients: "+str(linear_model.coefficients))
print("Intercept: "+ str(linear_model.intercept))

Coefficients: [1.0427632763748864e-19,1.0000000000000002,-1.455893291150934e-18,-2.6721362977261686e-19,4.840545498327886e-19,-9.688936960677006e-19]
Intercept: -1.918021604029193e-18


In [126]:
trainSummary=linear_model.summary
print('RMSE : %f' %trainSummary.rootMeanSquaredError)
print('r2: %f' % trainSummary.r2adj)

RMSE : 0.000000
r2: 1.000000


In [127]:
predictions=linear_model.transform(test_df)

In [128]:
predictions.show()

+--------------------+--------+--------------------+
|            features|Survived|          prediction|
+--------------------+--------+--------------------+
|(6,[0,2],[180.0,3...|     0.0|-3.55604411107148...|
|(6,[0,2],[414.0,2...|     0.0|-1.98636811716576...|
|(6,[0,2],[675.0,2...|     0.0|2.522975339621877...|
|(6,[0,2],[733.0,2...|     0.0|3.12777803991931E-17|
|(6,[0,2],[823.0,3...|     0.0|2.857745097788845...|
|[1.0,0.0,22.0,1.0...|     0.0|-4.11350906079757...|
|[3.0,1.0,26.0,0.0...|     1.0|  1.0000000000000002|
|[4.0,1.0,35.0,1.0...|     1.0|  1.0000000000000002|
|[5.0,0.0,35.0,0.0...|     0.0|-6.01524995942712...|
|[6.0,0.0,29.69911...|     0.0|-5.27263029908479...|
|[12.0,1.0,58.0,0....|     1.0|  1.0000000000000002|
|[13.0,0.0,20.0,0....|     0.0|-3.74798896059073...|
|[15.0,0.0,14.0,0....|     0.0|-2.8346267523093E-17|
|[20.0,1.0,29.6991...|     1.0|  1.0000000000000002|
|[21.0,0.0,35.0,0....|     0.0|-7.58757200116848...|
|[37.0,1.0,29.6991...|     1.0|  1.00000000000